In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import re
#from scipy.interpolate import griddata
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
#from dedalus import public as de

In [2]:
def getproj(z, r_inner, r_outer):    
    if np.abs(z) < r_inner: 
        start_point = np.sqrt(r_inner**2 - z**2)
    else:
        start_point = 0
    end_point = np.sqrt(r_outer**2 - z**2)
    gridpoints = int(rscale.shape[0]/(r_outer-r_inner) * (end_point-start_point))
    
    #finding equidistant point with the same density as the original grid along the path of interpolation
    z_desired = np.asarray(np.linspace( start_point, end_point, num = gridpoints, endpoint = False))
    y_coord = np.asarray([z for _ in z_desired])
    inter_coord = np.asarray([z_desired, y_coord]).T
    
    #padding and mesh
    phi_pad = np.append(phiscale[:], 2*np.pi)
    s_pad   = np.append(z_desired[:], end_point)
    s_mesh, phi_mesh = np.meshgrid(s_pad, phi_pad)
    
    return inter_coord, s_mesh, phi_mesh

In [3]:
def getplotdata(data, xykeys, datakeys):
    # this function creates a dictionary so I can find points more relevant with their coordinates
    datapoints = data.reshape(data.shape[0],-1)
    datadic = dict(zip(xykeys, datapoints.T))
    datas = np.asarray([datadic[key] for key in datakeys]).T
    return datas

In [4]:
def plotnsave(val, s_mesh, phi_mesh, mydir, y):
    fig, ax = plt.subplots(subplot_kw=dict(polar=True))
    pcm = ax.pcolormesh(phi_mesh, s_mesh, val, cmap='RdYlBu_r',vmin = vmin, vmax = vmax)
    ax.set_xticks([])
    ax.set_axis_off()
    ax.set_rlim([0, r_outer])
    cbar_ax = fig.add_axes([0.8, 0.2, 0.02, 0.6])
    cb = fig.colorbar(pcm, cax=cbar_ax)
    #these three line draws the boundaries, because axis_off
    ones = np.ones(100)
    angles = np.linspace(0, 2*np.pi,100)
    style = (0,(4,10))
    if s_mesh[1,0]!= 0:
        ax.fill_between(angles, s_mesh[1,0]*ones, y2 =s_mesh[1,0]*ones, fc='w')
    ax.fill_between(angles, r_inner*ones, y2 =r_inner*ones, fc='none', ec = 'k', linestyle = style)
    ax.fill_between(angles, s_mesh[1,-1]*ones, y2 =s_mesh[1,-1]*ones, fc='none', ec = 'k', linestyle = '-')
    ax.fill_between(angles, r_outer*ones, y2 =r_outer*ones, fc='none', ec = 'k', linestyle = style, linewidth=2.0)
    
    cb.set_label(plotvar, rotation=0)

    plt.savefig(mydir  + '/z%.3f_plot/%s_%04i' %(y ,"plot".format(0), counter) + '_' + plotvar + '.png', dpi=150)
    plt.close(fig)

In [37]:
#    if data.ndim == 5: np.linalg.norm(data, axis=4)
#if i want better performance:
#google: speedup scipy griddata for multiple interpolations between two irregular grids
def makeplots(mydir, r_inner, r_outer, y):
    #maybe not the best way, but I got lazy, and didn't want to not wrap everything in a function
    global files, data, rscale, thetascale, phiscale, counter, s_mesh, phi_mesh
    
    files = []
    for file in os.listdir(mydir):
        if file.endswith(".h5"):
            files.append(os.path.join(mydir, file))
    files.sort(key=lambda f: int(re.sub('\D','', f)))
    
    if cutheadtail:
        files = files[5:-2]
    
    counter = 0
    zdata = []
    
    for file in files:
        df = h5py.File(file,"r")
        
        dname = 'tasks/' + plotvar
        dataset = df[dname]
        data = np.asarray(dataset, dtype = np.float64)
        if data.ndim == 5:
            data = data[:,0,:,:,:]
        
        if counter == 0:
            
            rscale = dataset.dims[3][0][:]
            thetascale = dataset.dims[2][0][:]
            phiscale = dataset.dims[1][0][:]

            inter_coord, s_mesh, phi_mesh = getproj(y, r_inner, r_outer)

            xygrid = np.asarray([[r*np.sin(theta), r*np.cos(theta)] for theta in thetascale for r in rscale])
            inbetween = np.asarray([xy for xy in xygrid if (y - 1/26) <= xy[1] <= (y + 1/26)])
            # inbetween contains the relevent points. the smaller the set the faster the plot
            
            if method == 'fastlinear' or method == 'fastcubic':
                tri = Delaunay(inbetween) #triangulation that speeds up the process
            
            xykeys = list(map(str,xygrid))
            datakeys = list(map(str, inbetween))
            
            plot_dir = mydir + '/z%.3f_plot' %(y)
            if not os.path.exists(plot_dir):
                os.makedirs(plot_dir)


        for dats in data:
            datas = getplotdata(dats, xykeys, datakeys)

            if method == 'fastlinear':
                interpolators = [LinearNDInterpolator(tri, dat) for dat in datas]
                grid_z0 = np.asarray([interpolator(inter_coord) for interpolator in interpolators])
            elif method == 'fastcubic':
                interpolators = [CloughTocher2DInterpolator(tri, dat) for dat in datas]
                grid_z0 = np.asarray([interpolator(inter_coord) for interpolator in interpolators])
            else:
                grid_z0 = np.asarray([griddata(inbetween, dat, inter_coord, method = method) for dat in datas])
            
            if remove_m0:
                grid_z0 -= np.mean(grid_z0, axis=0)

            if saveplots:
                plotnsave(grid_z0, s_mesh, phi_mesh, mydir, y)

            if getData:
                zdata.append(grid_z0)
                
            counter += 1
            
    if getData:
        return zdata

In [38]:
mydir = "data_Ek5e-6Ra300_L191N63_init5max10e-7-u"
r_inner = 7/13.
r_outer = 20./13.
zs = np.linspace(-r_outer, r_outer, 66)[1:-1]
#zs = np.geomspace((r_outer+1)**2,1, 31)**(1/2)-1 not complete thought
plotvar = 'u'
remove_m0 = False
saveplots = False
getData = True
cutheadtail = True
method = 'fastlinear'
#vmin = -300 
#vmax = 300
alldata =[]
for z in zs:
    alldata.append(makeplots(mydir, r_inner, r_outer, z))

In [29]:
mydir = "data_Ek5e-6Ra300_L191N63_init5max10e-7-u"
r_inner = 7/13.
r_outer = 20./13.
zs = np.linspace(-r_outer, r_outer, 66)[1:-1]
#zs = np.geomspace((r_outer+1)**2,1, 31)**(1/2)-1 not complete thought
plotvar = 'u'
remove_m0 = False
saveplots = False
getData = True
cutheadtail = True
method = 'fastlinear'
#vmin = -300 
#vmax = 300
alldata = []
for z in zs:
    alldata.append(makeplots(mydir, r_inner, r_outer, z))

In [9]:
alldata = [np.array(data) for data in alldata]

In [39]:
[np.array(data).shape for data in alldata]

[(340, 384, 64)]

In [45]:
np.array(alldata[0]).shape

(340, 384, 64)

In [28]:
test

array([1.53846154, 1.46084863, 1.38560873, 1.31266927, 1.24195991,
       1.17341248, 1.10696087, 1.042541  , 0.98009076, 0.91954992,
       0.8608601 , 0.80396472, 0.74880889, 0.69533944, 0.64350481,
       0.59325501, 0.54454159, 0.49731757, 0.45153742, 0.40715698,
       0.36413346, 0.32242538, 0.28199252, 0.24279588, 0.20479767,
       0.16796124, 0.13225108, 0.09763276, 0.06407288, 0.03153908,
       0.        ])

In [24]:
test[1]-test[0]

0.031539080772679196

In [69]:
mydir = "data_Ek5e-6Ra300_L191N63_init5max10e-7-u"
r_inner = 7/13.
r_outer = 20./13.
#z = r_inner + factor*(r_outer-r_inner)
z = 0
plotvar = 'T'
remove_m0 = False
saveplots = False
getData = True
cutheadtail = True
alldata = []
vmin = -0.1
vmax = 0.1
method = 'fastlinear'
%time makeplots(mydir, r_inner, r_outer, z)

CPU times: user 1min 25s, sys: 13.6 s, total: 1min 38s
Wall time: 1min 38s


In [63]:
mydir = "data_Ek5e-6Ra300_L191N63_init5max10e-7-u"
r_inner = 7/13.
r_outer = 20./13.
#z = r_inner + factor*(r_outer-r_inner)
z = 0
plotvar = 'T'
remove_m0 = False
saveplots = False
getData = True
cutheadtail = True
alldata = []
vmin = 0
vmax = 1
method = 'fastlinear'
%time makeplots(mydir, r_inner, r_outer, z)

CPU times: user 2min 46s, sys: 15.8 s, total: 3min 2s
Wall time: 3min 1s


In [36]:
files

['data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s6.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s7.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s8.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s9.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s10.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s11.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s12.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s13.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s14.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5max10e-7-u_s15.h5',
 'data_Ek5e-6Ra300_L191N63_init5max10e-7-u/data_Ek5e-6Ra300_L191N63_init5m